In [1]:
import mediapipe as mp
import cv2
import numpy as np
import phoenix as px
from opentelemetry import trace
from phoenix.otel import register

/Users/priyanka./Documents/Mediapipe/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Setup Phoenix tracing
px.launch_app(port=6006)
tracer_provider = register(
    project_name="pushup-app",
    endpoint="http://localhost:6006/v1/traces",
    batch=False
)
trace.set_tracer_provider(tracer_provider)
tracer = trace.get_tracer(__name__)

❗️ The launch_app `port` parameter is deprecated and will be removed in a future release. Use the `PHOENIX_PORT` environment variable instead.


/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_cumulative_llm_token_count_total
  next(self.gen)
/opt/homebrew/Cellar/python@3.11/3.11.13/Frameworks/Python.framework/Versions/3.11/lib/python3.11/contextlib.py:144: SAWarning: Skipped unsupported reflection of expression-based index ix_latency
  next(self.gen)
Overriding of current TracerProvider is not allowed


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: pushup-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
# Mediapipe pose setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

I0000 00:00:1756732065.830997 4874411 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1756732065.964797 4875195 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756732065.985403 4875200 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
# Angle calculation utility (fixed)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [5]:
counter = 0
stage = None

def process_frame_pipeline(frame):
    global counter, stage

    # Recolor BGR to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # Pose detection
    results = pose.process(image_rgb)

    # Prepare output dictionary for tracing attributes
    pipeline_attrs = {
        "landmarks_detected": False,
        "shoulder": None,
        "elbow": None,
        "wrist": None,
        "angle": None,
        "counter": counter,
        "stage": stage,
    }

    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    try:
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            pipeline_attrs["landmarks_detected"] = True
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            pipeline_attrs["shoulder"] = shoulder
            pipeline_attrs["elbow"] = elbow
            pipeline_attrs["wrist"] = wrist

            angle = calculate_angle(shoulder, elbow, wrist)
            pipeline_attrs["angle"] = angle

            # Visualize angle on image
            cv2.putText(image_bgr, str(int(angle)),
                        tuple(np.multiply(elbow, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Push-up counter logic
            if angle > 160:
                stage = "up"
            if angle < 70 and stage == "up":
                stage = "down"
                counter += 1
                print(f"Push-up Count: {counter}")

            pipeline_attrs["counter"] = counter
            pipeline_attrs["stage"] = stage

        # Render push-up counter UI on image
        cv2.rectangle(image_bgr, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image_bgr, 'Push-ups', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image_bgr, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Draw pose landmarks
        mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    except Exception as e:
        pipeline_attrs["error.message"] = str(e)

    return image_bgr, pipeline_attrs



In [6]:
def main(video_path):
    cap = cv2.VideoCapture(video_path)
    global counter, stage
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Create a single span per frame pipeline
        with tracer.start_as_current_span("pushup_pipeline") as span:
            image, attrs = process_frame_pipeline(frame)
            for key, val in attrs.items():
                # Flatten lists for attribute storage
                if isinstance(val, list):
                    val = [float(x) for x in val]
                span.set_attribute(key, val)

        cv2.imshow('Push-up Counter', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [7]:
video_path = "/Users/priyanka./Documents/Mediapipe/push-up_1.mp4"
main(video_path)

W0000 00:00:1756732145.430966 4875195 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Push-up Count: 1
Push-up Count: 2
Push-up Count: 3
